<a href="https://colab.research.google.com/github/Rhezaaa/coursegit/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(trainX, trainY, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')
print('Done')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


[(['hallo'], 'A'), (['hai'], 'A'), (['hey'], 'A'), (['prosedur', 'donor', 'darah'], 'B'), (['bagaimana', 'cara', 'donor', 'darah'], 'B'), (['donor', 'darah'], 'B'), (['cara', 'donor', 'darah'], 'B'), (['prosedur', 'donor', 'darah'], 'B'), (['langkah-langkah', 'donor', 'darah'], 'B'), (['prosedur', 'permintaan', 'darah'], 'C'), (['bagaimana', 'cara', 'minta', 'darah'], 'C'), (['minta', 'darah'], 'C'), (['cara', 'minta', 'darah'], 'C'), (['prosedur', 'permintaan', 'darah'], 'C'), (['langkah-langkah', 'minta', 'darah'], 'C'), (['cara', 'dapat', 'darah'], 'C'), (['Apakah', 'dengan', 'Donor', 'Darah', 'dapat', 'menyehatkan', '?'], 'D'), (['apa', 'donor', 'sehat', '?'], 'D'), (['sehat', 'kah', 'donor', '?'], 'D'), (['Mengapa', 'harus', 'donor', 'darah', '?'], 'E'), (['kenapa', 'harus', 'donor', '?'], 'E'), (['harus', 'donor', '?'], 'E'), (['haruskah', 'donor', 'darah', '?'], 'E'), (['Apa', 'syarat-syarat', 'untuk', 'mendonorkan', 'darah', '?'], 'F'), (['syarat', 'donor'], 'F'), (['apa', 'sya

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
6/6 [==============================] - 1s 5ms/step - loss: 2.1154 - accuracy: 0.1333
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 2.1104 - accuracy: 0.1333
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 1.9868 - accuracy: 0.3000
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 1.9174 - accuracy: 0.2333
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 1.7852 - accuracy: 0.3667
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 1.7674 - accuracy: 0.3667
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 1.6446 - accurac

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  `tf.saved_model.SaveOptions` object that specifies SavedModel


In [31]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model("chatbot_model.h5")

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print("HALLO INI CHATBOT")

while True:
    message = input("You: ")

    if message.lower() == 'exit':
        print("Bot: Goodbye!")
        break

    ints = predict_class(message)
    res = get_response(ints, intents)
    print("Bot:", res)

HALLO INI CHATBOT
You: hai


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 110ms/step
Bot: Hai
You: exit
Bot: Goodbye!


In [34]:
%shell tensorflowjs_converter --input_format keras /content/chatbot_model.h5 /content/tfjs_model

2023-12-13 08:17:02.329082: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 08:17:02.329189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 08:17:02.339533: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 08:17:04.634081: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
